## Goal: Fit a time series model to the historical bitcoin daily closing values

1. [Simple Exponential Smoothing](#optparam1)
2. [Holt's Linear Trend Method](#optparam2)
3. [Damped Trend Method](#optparam3)
4. [Taxonomy of Exponential Smoothing Methods](#optparam4)


* Importing 'forecast' package which is very useful to model time series data
* As well as lattice package to use xyplot() function which is a simple function to plot time series in a neat way

In [ ]:
library(forecast)
library(lattice)
list.files(path = "../input")

In [ ]:
train = read.csv("../input/bitcoin_price_Training - Training.csv", header=T)
test = read.csv("../input/bitcoin_price_1week_Test - Test.csv", header=T)

# glimpse of top few rows of train data

head(train)

* We will predict the daily Closing value time series for future periods.
* The future data has for validation purposes in in the test data frame

In [ ]:
head(test)

* Creating **close_tr_df** as a data frame having dates and closing values in USD for **TRAIN** data
* Creating **close_val_df** as a data frame having dates and closing values in USD for **TEST** data

* We will also use the **mdy()** function from lubridate to convert Date column from factor to a POSIXCt object

In [ ]:
suppressPackageStartupMessages(library(ggplot2))
suppressPackageStartupMessages(library(dplyr))
suppressPackageStartupMessages(library(lubridate))

train$Date <- mdy(train$Date)
test$Date <- mdy(test$Date)

close_tr_df <- train[,c("Date", "Close")] %>% arrange(Date)
close_val_df <- test[,c("Date", "Close")] %>% arrange(Date)
tail(close_tr_df)
head(close_val_df)

* We can see that train data ends at July 31st 2017 while test data starts at August 1st 2017
* Lets see if we have any missing values

In [ ]:
table(is.na(close_tr_df))
table(is.na(close_val_df))

* No missing values
'
* Now using **ts() function and its START= and END=** arguments to create a **ts** object for TRAIN data and VALIDATION (TEST) DATA
* **ts** object is a time series object. We can do many flexible time series operations directly on this object
* We also plot the created ts object to see the pattern

In [ ]:
library(repr)
close_tr_ts <- ts(close_tr_df$Close,
                  start = c(as.numeric(format(close_tr_df$Date[1], "%Y")), as.numeric(format(close_tr_df$Date[1], "%j"))),
                  end = c(as.numeric(format(close_tr_df$Date[nrow(close_tr_df)], "%Y")), as.numeric(format(close_tr_df$Date[nrow(close_tr_df)], "%j"))),
                  frequency = 365)
close_val_ts <- ts(close_val_df$Close,
                  start = c(as.numeric(format(close_val_df$Date[1], "%Y")), as.numeric(format(close_val_df$Date[1], "%j"))),
                  end = c(as.numeric(format(close_val_df$Date[nrow(close_val_df)], "%Y")), as.numeric(format(close_val_df$Date[nrow(close_val_df)], "%j"))),
                  frequency = 365)
options(repr.plot.width=7 , repr.plot.height=7)
xyplot(close_tr_ts)

***
* There is a trend starting slowly from late 2015, increasing slowly till mid 2017 and very rapidly thereafter.
* Seasonality seems to be absent
* Non-stationary time series.

* **Forecasting approach from initial observations:**
    1. First we will start off by trying extremely basic models like naive forecast and drift.
    2. Then we will try out various exponential smoothing models starting with simple exponential smoothing and going on to ets with trend.
    3. We will also try ARIMA at the end.


In [ ]:
naive_close <- naive(close_tr_ts, h = nrow(close_val_df))
drift_close <- rwf(close_tr_ts, h = nrow(close_val_df), drift = T)

cat("1 step Naive model")
accuracy(naive_close, close_val_ts)
cat("Drift model i.e. First observation - Last observation")
accuracy(drift_close, close_val_ts)

* Both the models are yielding almost similar results.
* We that see that we are heavily overfitting the train data for both models. This is because the right at the end of the train time series, there is a severe trend and the whole range of the time series has changed.
* This provides more evidence that exponential smoothing models might provide a reasonable estimate as we can weight the recent observations more in that.
* Let us try **Exponential Smoothing models**
<a id="optparam1"></a>

## 1. Simple Exponential Smoothing


In [ ]:
close_ses <- ses(close_tr_ts)
summary(close_ses)

* **ses()** function minimizes the SSE to estimate the right value of ALPHA and INITIAL STATE "L0"
* Looking at the summary, the ideal values for ALPHA and L0 are selected as 0.9798 and 134.2529 respectively
* Lets look at our performance on test data

In [ ]:
close_ses_preds = data.frame(predict(close_ses, h= nrow(close_val_df)))
accuracy(close_ses_preds$Point.Forecast, close_val_ts)

* The model is still not performing efficiently. RMSE for test set is still $ 352 which was nearly the same as the one obtained by Naive model
* Factors like optimal **Alpha being 0.97** & **Naive model performing equally well as exponential smoothing** tends to suggest that the **observations in the recent past are more important when predicting Bitcoin Closing values**

* Note that we still haven't accomodated a model component for trend.
* We can definitely see that trend starts from late 2015 and is pretty much persistent throughout until the end
* So let us try to accomodate this trend
<a id="optparam2"></a>

## 2. Holt's linear trend method

In [ ]:
#autoplot(close_ses) +
#  autolayer(fitted(close_ses), series="Fitted") +
#  ylab("Bitcoin Daily Closing in $") + xlab("Year")

close_holt_lt <- holt(close_tr_ts)
summary(close_holt_lt)

In [ ]:
close_holt_lt_preds = data.frame(predict(close_holt_lt, h= nrow(close_val_df)))
accuracy(close_holt_lt_preds$Point.Forecast, close_val_ts)

* RMSE has reduced by $ 30

* In Holt's linear trend method, all the future forecasts are either trended up or trended down constantly.
* This is not practical for most real-life datasets since some time or the other in the future, the trend will flatten.
* To accomodate this, there is a method called as DAMPED LINEAR TREND
* DAMPED LINEAR TREND method is very similar to Holt's method just with an additional dampening parameter which slowly reduces the trend of forecasts.
* Short term forecasts are trended in this method and long-term forecasts are flattened.
<a id="optparam3"></a>

## 3. Damped Trend Method

In [ ]:
close_holt_damped <- holt(close_tr_ts, damped=TRUE, phi=0.8)
close_holt_damped

# The following code is a plot of Time series with fitted values and is weirdly not working in Kaggle Kernel

#autoplot(close_tr_ts) +
#  autolayer(close_holt_lt, series="Holt's method", PI=FALSE) +
#  autolayer(close_holt_damped, series="Damped Holt's method", PI=FALSE) +
#  ggtitle("Forecasts from Holt's method") + xlab("Year") +
#  ylab("Closing value of Bitcoin in USD") +
#  guides(colour=guide_legend(title="Forecast"))

In [ ]:
close_holt_damped_preds = data.frame(predict(close_holt_damped, h= nrow(close_val_df)))
accuracy(close_holt_damped_preds$Point.Forecast, close_val_ts)

* RMSE did not improve at all for Point.Forecast
* Interestingly, I tried checking the accuracy with Upper 95% Confidence boundary and the RMSE reduced extensively.
* This suggests that our model is under-estimating the trend.
* Lets see if we can correct this in future models

In [ ]:
accuracy(close_holt_damped_preds$Hi.95, close_val_ts)

<a id="optparam4"></a>

## 4. Taxonomy of Exponential Smoothing Methods

* There are multiple combinations of Exponential Smoothing models available to us.
* Models with trend, seasonal and level components with either of the components absent/present/additive/multiplicative.
* **ets()** function takes in a 'ts' object and estimates the best component driven Exponential Smoothing method among several options available

In [ ]:
close_ets <- ets(close_tr_ts, damped=FALSE, allow.multiplicative.trend=TRUE)
close_ets

In [ ]:
close_ets_preds = data.frame(predict(close_ets, h= nrow(close_val_df)))
accuracy(close_ets_preds$Point.Forecast, close_val_ts)

* Finally a BIG improvement w.r.t Point.Forecast
* Our last best RMSE for point forecast was $ 328
* ets() chose an Exponential Smoothing model with Multiplicative Level, Multiplicative Trend and Seasonality Absent.
* Note that we had set the trend dampening to be FALSE, let us set it to be true, although intuitively is should not help because our previous model was under-estimating the values.
* Before that lets quickly check the RMSE with Upper 95 CL as our forecasts

In [ ]:
accuracy(close_ets_preds$Hi.95, close_val_ts)

* RMSE has increased for Upper 95 CL. This is good. This means our model has improved and not under-estimated the forecasts.
* Although, I soon found out that there is still some under estimation as Upper 80 CL gives an RMSE below 200.
* This might be an indication that in the validation data we have, the values are slightly inflated. If we use Higher 80 CL, it might indicate an overfit of the current validation data.
* So sticking to Point Forecasts as our actual forecasts will still be a better decision.

In [ ]:
accuracy(close_ets_preds$Hi.80, close_val_ts)

* Search for best Component Exponential Smoothing model with **trend DAMPNING**
* As expected, the model does not help much.

In [ ]:
close_ets_damped <- ets(close_tr_ts, damped=TRUE, allow.multiplicative.trend=TRUE)
close_ets_damped_preds = data.frame(predict(close_ets_damped, h= nrow(close_val_df)))
accuracy(close_ets_damped_preds$Point.Forecast, close_val_ts)

<a id="optparam5"></a>

## 5. ARIMA Modelling

* So the best results thus far has been ETS(M,M,N) Model - $ 281 RMSE.
* Let us try to better it using ARIMA models.
* ARIMA models use the past observations and past errors to create extremely flexible forecasting systems
* We will use a non-seasonal ARIMA model
    

In [ ]:
(close_arima <- auto.arima(close_tr_ts))

In [ ]:
close_arima_preds = close_arima %>% forecast(h= nrow(close_val_df)) %>% data.frame()
accuracy(close_arima_preds$Point.Forecast, close_val_ts)

* We can see that AUTO.ARIMA() selected an ARIMA model with p = 3, d = 2 and q = 0.
* RMSE is also the best we have had so far, $ 220.

#### Next we will try to better this if it is possible



## To be continued...